In [64]:
import pandas as pd
import numpy as np
import time
import os
from os import listdir
from os.path import isfile, join
from ast import literal_eval

In [39]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:97.5% !important; }</style>"))

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# Objective of this file:

1. Label each tweet as a CT tweet or not (Naive approach)
    * Contains Hashtags?
    * Contains Links?

## Hashtag and Link Lists

In [40]:
general_conspiracy_hashtags = [
    'plandemic',
    'scamdemic',
    'covidhoax',
    'nwo',
    'covid1984',
    'plandemia',
    'agenda21',
    'thegreatreset',
    'agenda2030',
    'newworldorder',
    'wakeupamerica',
#     'wakeup',
    'openamericanow',
    'firefauci',
    'wwg1wga',
    'qanon',
    'coronahoax'
]

keywords = [
    'plandemic',
    'scamdemic',
    'covidhoax',
    'covid hoax',
    'covid1984',
    'plandemia',
    'new world order',
    'wake up america',
    'open america now',
    'fire fauci',
    'wwg1wga',
    'qanon',
    'coronahoax',
    'corona hoax',
]

# CHANGE THESE LATER
CT_link_list = ['zerohedge.com', 'infowars.com', 'principia-scientific.com',
'tx.voice-truth.com', 'humansarefree.com', 'activistpost.com'
'gnews.org', 'wakingtimes.com', 'brighteon.com','thewallwillfall.org','sott.net',]


## Columns to aggregate

In [41]:
link_tag_keyword_cols = []

for link in CT_link_list:
    link_tag_keyword_cols.append(f'Link - {link}')      
for hashtag in general_conspiracy_hashtags:
    link_tag_keyword_cols.append(f'Tag - {hashtag}')
for keyword in keywords:
    link_tag_keyword_cols.append(f'Keyword - {keyword}')


# IF I WANT DUMMIES BROKEN DOWN BY ('POST', 'RETWEET', 'TOTAL')!!!!!
# for link in CT_link_list:
#     for tweet_type in ('POST', 'RETWEET', 'TOTAL'):
#         link_tag_keyword_cols.append(f'Link - {link} - {tweet_type}')      
# for hashtag in general_conspiracy_hashtags:
#     for tweet_type in ('POST', 'RETWEET', 'TOTAL'):
#         link_tag_keyword_cols.append(f'Tag - {hashtag} - {tweet_type}')


In [72]:
def hashtag_in_list(list_of_hashtags_in_tweet):
    return any(hashtag.upper() in [tag.upper() for tag in list_of_hashtags_in_tweet] for hashtag in general_conspiracy_hashtags)

# Final Function

In [99]:
def add_CT_dummies(READ_PATH, SAVE_PATH):
    
    df = pd.read_csv(READ_PATH, parse_dates=['date'], converters={'hashtags': eval})#.set_index('retweet')
#     pd.read_csv(df_name, converters={'hashtags': eval})
    del df['Unnamed: 0']
    
#     df['hashtags'] = df['hashtags'].apply(literal_eval)
    
    # create dummies for each hashtag and link (in each tweet)    
    for link in CT_link_list:
        df[f'Link - {link}'] = df['urls'].str.contains(link, case=False).astype(int)
        
    for tag in general_conspiracy_hashtags:
        df[f'Tag - {tag}'] = df['hashtags'].apply(hashtag_in_list).astype(int)
#         df[f'Tag - {tag}'] = df['hashtags'].apply(literal_eval).apply(hashtag_in_list).astype(int)
        
        
    for keyword in keywords:
        df[f'Keyword - {keyword}'] = df['tweet'].str.contains(keyword, case=False).astype(int)

    df.insert(loc=0, column='CT Tweet (Dummy)', value=(df[link_tag_cols].sum(axis=1) > 0).astype(int))
    
    
    # somewhat unrelated - fix broken 'retweet' boolean column
    df['retweet'] = df['tweet'].str.startswith('RT').astype(int)
    
    
    return df
        
#     df.to_csv(SAVE_PATH, index=False)

In [43]:
test = pd.read_csv(TEST_PATH)

In [45]:
test.head(1)

,Unnamed: 0,id,conversation_id,date,place,tweet,language,hashtags,user_id,username,urls,retweet,nlikes,nreplies,nretweets,near,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date
0,0,1111427090564304897,1111427090564304897,2019-03-28 17:37:22,NaN,Swalwell TDS 2 years &amp; counting.,1.0,[],3315840117,alwaysberunning,[],False,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
test.loc[189]

Unnamed: 0                                                                                        189
id                                                                                1108057508608802816
conversation_id                                                                   1108057508608802816
date                                                                              2019-03-19 10:27:51
place                                                                                             NaN
tweet              @realDonaldTrump @POTUS @FLOTUS #WRWY #WWG1WGA #TRUMP2020  https://t.co/6DHji50kt9
language                                                                                            0
hashtags                                                             ['wrwy', 'wwg1wga', 'trump2020']
user_id                                                                                    3315840117
username                                                                          

In [88]:
%%time

TEST_PATH = r"C:/Users/crackcocaine69xxx/Python Stuff/594/Twint/Looking Up All Conspiracy Hashtag User Tweets/All Conspiracy Tweeters' Tweets/Split 5/1140881537283633153_TWEETS.csv"

# TEST_PATH2 = r"C:/Users/crackcocaine69xxx/Python Stuff/594/Twint/Looking Up All Conspiracy Hashtag User Tweets/All Conspiracy Tweeters' Tweets/Split 5/173413459_TWEETS.csv"

df = add_CT_dummies(TEST_PATH,0)

Wall time: 28.8 s


### Searching for links with str.contains

In [98]:
%%time 

df = add_CT_dummies(TEST_PATH,0)

Wall time: 17.2 s


In [100]:
%%time 

df = add_CT_dummies(TEST_PATH,0)

Wall time: 16.7 s


In [104]:
df[df['CT Tweet (Dummy)']==1]

,CT Tweet (Dummy),id,conversation_id,date,place,tweet,language,hashtags,user_id,username,urls,retweet,nlikes,nreplies,nretweets,near,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,Link - zerohedge.com,Link - infowars.com,Link - principia-scientific.com,Link - tx.voice-truth.com,Link - humansarefree.com,Link - activistpost.comgnews.org,Link - wakingtimes.com,Link - brighteon.com,Link - thewallwillfall.org,Link - sott.net,Tag - plandemic,Tag - scamdemic,Tag - covidhoax,Tag - nwo,Tag - covid1984,Tag - plandemia,Tag - agenda21,Tag - thegreatreset,Tag - agenda2030,Tag - newworldorder,Tag - wakeupamerica,Tag - openamericanow,Tag - firefauci,Tag - wwg1wga,Tag - qanon,Tag - coronahoax,Keyword - plandemic,Keyword - scamdemic,Keyword - covidhoax,Keyword - covid hoax,Keyword - covid1984,Keyword - plandemia,Keyword - new world order,Keyword - wake up america,Keyword - open america now,Keyword - fire fauci,Keyword - wwg1wga,Keyword - qanon,Keyword - coronahoax,Keyword - corona hoax
189,1,1108057508608802816,1108057508608802816,2019-03-19 10:27:51,NaN,@realDonaldTrump @POTUS @FLOTUS #WRWY #WWG1WGA #TRUMP2020 https://t.co/6DHji50kt9,0.0,"[wrwy, wwg1wga, trump2020]",3315840117,alwaysberunning,[],0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
213,1,1106406260365099009,1106406260365099009,2019-03-14 21:06:23,NaN,@codeofvets #WWG1WGA 🇺🇸🇺🇸🇺🇸🇺🇸🇺🇸🇺🇸🇺🇸🇺🇸🇺🇸🇺🇸,0.0,[wwg1wga],3315840117,alwaysberunning,[],0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
446,1,1103053083167215616,1103053083167215616,2019-03-05 14:02:03,NaN,Keep your foot on the gas Patriots. #WWG1WGA,1.0,[wwg1wga],3315840117,alwaysberunning,[],0,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
460,1,1102824618170101760,1102824618170101760,2019-03-04 22:54:13,NaN,@realDonaldTrump @Realpersonpltcs More voter fraud coming your way America!! #WWG1WGA,1.0,[wwg1wga],3315840117,alwaysberunning,[],0,3,0,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
689,1,1097168391192768513,1097168391192768513,2019-02-17 08:18:23,NaN,@realDonaldTrump #thebestcloser #CloserNation #MAGA #KAG2020 #QAnon #WWG1WGA,0.0,"[thebestcloser, closernation, maga, kag2020, qanon, wwg1wga]",3315840117,alwaysberunning,[],0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0
699,1,1096631409479806978,1096631409479806978,2019-02-15 20:44:37,NaN,I’m so glad this Warrior is on my side. #WWG1WGA 🇺🇸,1.0,[wwg1wga],3315840117,alwaysberunning,[],0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
947,1,1091232486774820864,1091232486774820864,2019-01-31 23:11:13,NaN,Thank you. Be safe Sgt. Joe. #WWG1WGA 🙏,1.0,[wwg1wga],3315840117,alwaysberunning,[],0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
958,1,1091012513083285504,1091012513083285504,2019-01-31 08:37:07,NaN,#TRUMP2020 #WWG1WGA,0.0,"[trump2020, wwg1wga]",3315840117,alwaysberunning,[],0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1403,1,1082886034172432384,1082886034172432384,2019-01-08 22:25:24,NaN,We believe in you. Don’t give up. God loves you we love you. #WWG1WGA https://t.co/OQdgTyMBDX,1.0,[wwg1wga],3315840117,alwaysberunning,['https://twitter.com/yuri_bezmenov2/status/1082879238024454144'],0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [105]:
pd.concat([df.iloc[189:190], test.iloc[189:190]])

,CT Tweet (Dummy),id,conversation_id,date,place,tweet,language,hashtags,user_id,username,urls,retweet,nlikes,nreplies,nretweets,near,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,Link - zerohedge.com,Link - infowars.com,Link - principia-scientific.com,Link - tx.voice-truth.com,Link - humansarefree.com,Link - activistpost.comgnews.org,Link - wakingtimes.com,Link - brighteon.com,Link - thewallwillfall.org,Link - sott.net,Tag - plandemic,Tag - scamdemic,Tag - covidhoax,Tag - nwo,Tag - covid1984,Tag - plandemia,Tag - agenda21,Tag - thegreatreset,Tag - agenda2030,Tag - newworldorder,Tag - wakeupamerica,Tag - openamericanow,Tag - firefauci,Tag - wwg1wga,Tag - qanon,Tag - coronahoax,Keyword - plandemic,Keyword - scamdemic,Keyword - covidhoax,Keyword - covid hoax,Keyword - covid1984,Keyword - plandemia,Keyword - new world order,Keyword - wake up america,Keyword - open america now,Keyword - fire fauci,Keyword - wwg1wga,Keyword - qanon,Keyword - coronahoax,Keyword - corona hoax,Unnamed: 0
189,1.0,1108057508608802816,1108057508608802816,2019-03-19 10:27:51,NaN,@realDonaldTrump @POTUS @FLOTUS #WRWY #WWG1WGA #TRUMP2020 https://t.co/6DHji50kt9,0.0,"[wrwy, wwg1wga, trump2020]",3315840117,alwaysberunning,[],0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN
189,NaN,1108057508608802816,1108057508608802816,2019-03-19 10:27:51,NaN,@realDonaldTrump @POTUS @FLOTUS #WRWY #WWG1WGA #TRUMP2020 https://t.co/6DHji50kt9,0.0,"['wrwy', 'wwg1wga', 'trump2020']",3315840117,alwaysberunning,[],0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,189.0


# Apply Final Function

In [181]:
# folder containing folders of all split user lookups
OG_READ_PATH_ROOT = r"C:/Users/crackcocaine69xxx/Python Stuff/594/Twint/Looking Up All Conspiracy Hashtag User Tweets/All Conspiracy Tweeters' Tweets/"

# folder which will contain folders of all split user lookup AGGREGATIONS (weekly CT activity)
OG_WRITE_PATH_ROOT = r"C:/Users/crackcocaine69xxx/Python Stuff/594/Twint/Looking Up All Conspiracy Hashtag User Tweets/Labeled OG Tweets/"

og_splits = [5,6,7,8,9]

for split in og_splits:
    local_read_path_root = OG_READ_PATH_ROOT + f'Split {split}/'
    local_write_path_root = OG_WRITE_PATH_ROOT + f'Split {split}/'
    
    # make new folders to store processed tweets
    if not os.path.exists(local_write_path_root):
        os.makedirs(local_write_path_root)
    
    # find all users who have been searched and stored in this folder
    users_in_this_split = [int(f.split('_')[0]) for f in listdir(local_read_path_root) if isfile(join(local_read_path_root, f))]

    # aggregate each user's tweets
    for user in users_in_this_split:
        add_CT_dummies(READ_PATH=local_read_path_root+f'{user}_TWEETS.csv', SAVE_PATH=local_write_path_root+f'{user}_TWEETS_WITH_DUMMIES.csv')

KeyboardInterrupt: 